In [1]:
import numpy as np
import plotly
import plotly.graph_objs as go
import torch
from torch.utils.data import DataLoader
from experiment import Net, InitDataset, CreateModel, LoadCheckpoint
from yaml import safe_load
import torch.nn as nn

exphash = 'a1ecc37ef'

exp = 'results/%s' % exphash
with open(exp) as f:
  h = safe_load(f.read())
h = h['exp_info']  

ds = InitDataset(h)


model = CreateModel(h['hyperparameters'])
LoadCheckpoint(model, 'models/%s'%exphash)

model = model.cuda()


[15:00:44] Loaded model: <All keys matched successfully>


In [2]:
train_loader = DataLoader(ds, batch_size=512, shuffle=False)


In [3]:
ds[0]

{'char_labels': tensor([34, 55, 40, 43, 36,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        device='cuda:0'),
 'tok_labels': tensor(0, device='cuda:0'),
 'mask': tensor([ True,  True,  True,  True,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False],
        device='cuda:0'),
 'features': tensor([-1.6649e-02, -6.6612e-02, -1.6329e-02, -4.2112e-02, -8.0348e-03,
         -1.3965e-02, -6.3488e-02, -2.0491e-02, -8.5822e-03, -6.3427e-02,
         -2.8296e-02, -3.3587e-02, -3.5466e-02, -5.2275e-03, -2.0351e-02,
         -6.0686e-02, -5.0486e-02, -5.8112e-02, -2.1134e-02, -5.8061e-02,
         -3.6556e-02, -3.8286e-02, -9.5839e-03, -2.8228e-02, -1.0817e-01,
         -4.2421e-02, -6.7244e-03, -7.6137e-02,  1.3189e-02, -1.9380e-02,
         -3.8669e-02, -1.0872e-02, -1.7320e-02, -3.3488e-02, -6.0760e-02,
         -5.3773e-02, -3.7320e-02, -2.9963e-02, -5.9872e-02, -2.6235e-02,
         -5.3190e-02, -3.6603e-02, -7.2672e-02, -3.

In [4]:
ds[3000:3010]

{'char_labels': tensor([[61, 48, 58, 48, 59, 44, 43,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [52, 40, 59, 59,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [40, 42, 47, 48, 44, 61, 44, 43,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [43, 44, 45, 44, 53, 42, 44,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [48, 53, 59, 44, 57, 53, 40, 51,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [55, 44, 57, 58, 48, 40, 53,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [52, 44, 59, 47, 54, 43, 58,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [46, 48, 53, 46,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [40, 57, 57, 44, 58, 59, 44, 43,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [54, 59, 47, 44, 57, 62, 48, 58, 44,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
        device='cuda:0'),
 'tok_labels': tensor([3000, 3001, 3002, 3003, 3004, 3005, 3006, 3007, 3008, 3009],
        device='cuda:0'),
 'mask': tensor([[

In [16]:
xent = nn.CrossEntropyLoss()
def xent_loss_fn(output, labels):
  return xent(output.view(-1, output.size(-1)), labels.view(-1))

# returns char acc and word acc
def acc_fn(output, labels):
  top = output.argmax(-1)
  right = top==labels
  char_acc = right.float()
  word_acc = (right.sum(-1)==right.shape[-1]).float()
  return char_acc, word_acc


In [17]:
Torch2Py = lambda x: x.cpu().numpy().tolist()

In [21]:
def Validate(val_loader, model):
  cum_char_acc = 0
  cum_word_acc = 0
  cum_xent_loss = 0

  right_chars = []
  right_words = []
  losses_ = []
  maxes = []
  labels_ = []

  for i, data in enumerate(val_loader):
    inputs = data['features']
    labels = data['char_labels']
    labels_ += Torch2Py(labels)
    with torch.no_grad():
      outputs = model(inputs)
      maxes += Torch2Py(outputs.argmax(-1))
      xent_loss = xent_loss_fn(outputs, labels)
      cum_xent_loss += xent_loss

      char_acc, word_acc = acc_fn(outputs, labels)
      right_chars += Torch2Py(char_acc)
      right_words += Torch2Py(word_acc)
      cum_char_acc += char_acc.mean()
      cum_word_acc += word_acc.mean()

  steps = i+1
  print(cum_xent_loss/steps, cum_char_acc/steps, cum_word_acc/steps)
  return right_chars, right_words, maxes, labels_


In [22]:
right_chars, right_words, maxes, labels = Validate(train_loader, model)

tensor(0.0318, device='cuda:0') tensor(0.9895, device='cuda:0') tensor(0.8864, device='cuda:0')


In [23]:
len(right_chars), len(right_words), len(maxes), len(labels),

(25082, 25082, 25082, 25082)

In [24]:
idx_to_char_map= {v:k for k,v in ds.char_to_idx_map.items()}

In [25]:
def GetChars(encoded):
  s = ""
  for c in encoded:
    s+= idx_to_char_map[c]
  return s.replace('\x00','_')

In [29]:
for i, r in enumerate(right_words):
  if r==0 and i%1==0:
    print(GetChars(maxes[i]), GetChars(labels[i]))

[unused906]_______ [pad]_____________
[unused626]_______ [unused0]_________
[unused326]_______ [unused1]_________
[unused326]_______ [unused2]_________
[unused316]_______ [unused3]_________
[unused316]_______ [unused4]_________
[unused326]_______ [unused5]_________
[unused623]_______ [unused6]_________
[unused326]_______ [unused7]_________
[unused326]_______ [unused8]_________
[unused306]_______ [unused9]_________
[unused326]_______ [unused10]________
[unused326]_______ [unused11]________
[unused926]_______ [unused12]________
[unused616]_______ [unused13]________
[unused306]_______ [unused14]________
[unused336]_______ [unused15]________
[unused326]_______ [unused16]________
[unused326]_______ [unused17]________
[unused602]_______ [unused18]________
[unused926]_______ [unused19]________
[unused326]_______ [unused20]________
[unused326]_______ [unused21]________
[unused726]_______ [unused22]________
[unused396]_______ [unused23]________
[unused326]_______ [unused24]________
[unused326]_